In [1]:

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import scipy as sp
import statsmodels as sm
import jupyter as jp
import  sklearn as sk
import yellowbrick as yb




In [2]:

# read the processed_events_final file
df = pd.read_csv('processed_events_final.csv', delimiter='|')





/var/folders/0c/b8f44lfd3p73j8czwx0s68zw0000gn/T/ipykernel_33857/657840752.py:2: DtypeWarning: Columns (6,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('processed_events_final.csv', delimiter='|')


In [3]:
# check the first few rows of the dataframe
df.head()

,TIMESTAMP_EVENT,PRICE,PROPOSITION,USER_CLIENT_NUMBER,ARTICLE_CATEGORIE,USER_SESSION_ID,PROMOTION_LABEL,PAGE_NAME,PAGE_SECTION,PAGE_SECTION_POSITION,...,precipitation_coverage_lead_2,precipitation_amount_lead_2,temperature_lead_3,precipitation_coverage_lead_3,precipitation_amount_lead_3,temperature_lead_4,precipitation_coverage_lead_4,precipitation_amount_lead_4,total_spend_on_category_product,total_spend_on_product
0,2022-02-28T14:51:25.349Z,25.4760,554657,386730,"1,081,511,193",933,Actie,Shopping Cart,Recommended Products,2.0,...,0.0,0.0,2.9,0.0,0.0,1.8,0.0,0.0,0.0,0.0
1,2022-02-28T14:51:46.443Z,25.4760,554657,386730,"1,081,511,193",933,Actie,Shopping Cart,Recommended Products,2.0,...,0.0,0.0,2.9,0.0,0.0,1.8,0.0,0.0,0.0,0.0
2,2022-02-28T14:52:06.207Z,25.4760,554657,386730,"1,081,511,193",933,Actie,Shopping Cart,Recommended Products,2.0,...,0.0,0.0,2.9,0.0,0.0,1.8,0.0,0.0,0.0,0.0
3,2022-02-28T14:52:06.211Z,12.5268,349240,386730,"1,091,661,078",933,Actie,Shopping Cart,Recommended Products,1.0,...,0.0,0.0,2.9,0.0,0.0,1.8,0.0,0.0,0.0,0.0
4,2022-02-28T14:51:25.319Z,12.5268,349240,386730,"1,091,661,078",933,Actie,Shopping Cart,Recommended Products,1.0,...,0.0,0.0,2.9,0.0,0.0,1.8,0.0,0.0,0.0,0.0


In [4]:
# find the row of PROPOSITION = '802428'
df.loc[df['PROPOSITION'] == 802428]

,TIMESTAMP_EVENT,PRICE,PROPOSITION,USER_CLIENT_NUMBER,ARTICLE_CATEGORIE,USER_SESSION_ID,PROMOTION_LABEL,PAGE_NAME,PAGE_SECTION,PAGE_SECTION_POSITION,...,precipitation_coverage_lead_2,precipitation_amount_lead_2,temperature_lead_3,precipitation_coverage_lead_3,precipitation_amount_lead_3,temperature_lead_4,precipitation_coverage_lead_4,precipitation_amount_lead_4,total_spend_on_category_product,total_spend_on_product
423,2021-12-23T14:47:24.040Z,1.386,802428,749907,"1,031,161,221",41205,NaN,NaN,NaN,NaN,...,20.83,0.106,-5.9,0.00,0.000,2.5,58.33,1.145,0.0,0.0
424,2021-12-23T10:55:05.647Z,1.386,802428,749907,"1,031,161,221",41205,NaN,NaN,NaN,NaN,...,20.83,0.106,-5.9,0.00,0.000,2.5,58.33,1.145,0.0,0.0
431,2021-12-22T13:09:01.146Z,1.386,802428,749907,"1,031,161,221",59989,NaN,NaN,NaN,NaN,...,83.33,5.779,-5.2,20.83,0.106,-5.9,0.00,0.000,0.0,0.0
432,2021-12-22T09:57:03.676Z,1.386,802428,749907,"1,031,161,221",59989,NaN,NaN,NaN,NaN,...,83.33,5.779,-5.2,20.83,0.106,-5.9,0.00,0.000,0.0,0.0
433,2021-12-22T12:45:56.078Z,1.386,802428,749907,"1,031,161,221",59989,NaN,NaN,NaN,NaN,...,83.33,5.779,-5.2,20.83,0.106,-5.9,0.00,0.000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9935682,2022-03-03T10:04:53.198Z,1.848,802428,804960,"1,031,161,221",67631,NaN,Purchase List,Recommended Products,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
9935721,2022-01-10T11:24:33.876Z,1.914,802428,804960,"1,031,161,221",140232,NaN,Purchaselist,Recommended Products,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
9935722,2022-01-10T11:20:30.149Z,1.914,802428,804960,"1,031,161,221",140232,NaN,Purchaselist,Recommended Products,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
9935741,2021-12-31T10:43:48.939Z,1.650,802428,804960,"1,031,161,221",155020,NaN,Purchaselist,Recommended Products,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0


In [5]:
df.info(), df.shape, df.columns


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9935809 entries, 0 to 9935808
Data columns (total 42 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   TIMESTAMP_EVENT                  object 
 1   PRICE                            float64
 2   PROPOSITION                      int64  
 3   USER_CLIENT_NUMBER               int64  
 4   ARTICLE_CATEGORIE                object 
 5   USER_SESSION_ID                  int64  
 6   PROMOTION_LABEL                  object 
 7   PAGE_NAME                        object 
 8   PAGE_SECTION                     object 
 9   PAGE_SECTION_POSITION            float64
 10  PROMOTION_PRICE                  float64
 11  PRODUCT_TYPE                     object 
 12  DEVICE_INFO_BRAND                object 
 13  DEVICE_INFO_TYPE                 object 
 14  DEVICE_INFO_BROWSER              object 
 15  USER_SALES_GROUP                 object 
 16  USER_SEGMENT                     object 
 17  USER_SAL

(None,
 (9935809, 42),
 Index(['TIMESTAMP_EVENT', 'PRICE', 'PROPOSITION', 'USER_CLIENT_NUMBER',
        'ARTICLE_CATEGORIE', 'USER_SESSION_ID', 'PROMOTION_LABEL', 'PAGE_NAME',
        'PAGE_SECTION', 'PAGE_SECTION_POSITION', 'PROMOTION_PRICE',
        'PRODUCT_TYPE', 'DEVICE_INFO_BRAND', 'DEVICE_INFO_TYPE',
        'DEVICE_INFO_BROWSER', 'USER_SALES_GROUP', 'USER_SEGMENT',
        'USER_SALES_DISTRICT', 'USER_PROMOTIONS_ALLOWED', 'EVENT', 'DATE',
        'TIME', 'purchases_7_day_after', 'purchases_30_day_after', 'date',
        'temperature', 'precipcover', 'precip', 'temperature_lead_1',
        'precipitation_coverage_lead_1', 'precipitation_amount_lead_1',
        'temperature_lead_2', 'precipitation_coverage_lead_2',
        'precipitation_amount_lead_2', 'temperature_lead_3',
        'precipitation_coverage_lead_3', 'precipitation_amount_lead_3',
        'temperature_lead_4', 'precipitation_coverage_lead_4',
        'precipitation_amount_lead_4', 'total_spend_on_category_product',

### 1. Find the top-selling products (e.g top 5) based on the quantity sold of each of the products

In [6]:
# Function to recommend top-sold unique products for all clients
def recommend_top_sold_products(dataframe, n=5):
    # Filter out rows where PRICE is 0 to avoid division by zero
    dataframe = dataframe[dataframe['PRICE'] != 0]
    
    # Calculate total spend by each customer on each product
    total_spend_per_client_df = dataframe.groupby(['PROPOSITION', 'USER_CLIENT_NUMBER'])['total_spend_on_product'].max().reset_index()
    
    # Calculate total money spent on each product by all customers
    total_spend_df = total_spend_per_client_df.groupby('PROPOSITION')['total_spend_on_product'].sum().reset_index()
    
    # Merge total spend with price information
    price_df = dataframe[['PROPOSITION', 'PRICE']].drop_duplicates()
    total_spend_df = total_spend_df.merge(price_df, on='PROPOSITION')
    
    # Calculate total quantity sold for each product
    total_spend_df['quantity_sold'] = total_spend_df['total_spend_on_product'] / total_spend_df['PRICE']
    
    # Sort products by total quantity sold in descending order
    total_spend_df = total_spend_df.sort_values(by='quantity_sold', ascending=False)
    
    # Ensure unique products in the recommendation
    top_products = total_spend_df.drop_duplicates(subset=['PROPOSITION']).head(n)
    
    return top_products



### 2. recommend top-sold products based on the product category that the user buys most

In [7]:
# Function to recommend top-sold unique products based on the product category that the user buys most
def recommend_top_sold_products_by_category(dataframe, user_client_number, n=5):
    # Filter out rows where PRICE is 0 to avoid division by zero
    dataframe = dataframe[dataframe['PRICE'] != 0]
    
    # Find the product category that the user buys the most
    user_category = dataframe[dataframe['USER_CLIENT_NUMBER'] == user_client_number]['ARTICLE_CATEGORIE'].mode()[0]
    
    # Filter dataframe for the user's top category
    category_df = dataframe[dataframe['ARTICLE_CATEGORIE'] == user_category]
    
    # Calculate total spend by each customer on each product within the user's top category
    total_spend_per_client_df = category_df.groupby(['PROPOSITION', 'USER_CLIENT_NUMBER'])['total_spend_on_product'].max().reset_index()
    
    # Calculate total money spent on each product within the user's top category by all customers
    total_spend_df = total_spend_per_client_df.groupby('PROPOSITION')['total_spend_on_product'].sum().reset_index()
    
    # Merge total spend with price information
    price_df = category_df[['PROPOSITION', 'PRICE']].drop_duplicates()
    total_spend_df = total_spend_df.merge(price_df, on='PROPOSITION')
    
    # Calculate total quantity sold for each product within the user's top category
    total_spend_df['quantity_sold'] = total_spend_df['total_spend_on_product'] / total_spend_df['PRICE']
    
    # Sort products by total quantity sold in descending order
    total_spend_df = total_spend_df.sort_values(by='quantity_sold', ascending=False)
    
    # Ensure unique products in the recommendation
    top_products = total_spend_df.drop_duplicates(subset=['PROPOSITION']).head(n)
    
    return top_products



In [8]:
# Example of top-sold unique products recommendation for all clients
top_sold_products_all_clients = recommend_top_sold_products(df, 5)
print("Top Sold Products Recommendation for all clients:")
print(top_sold_products_all_clients)

# Example of category-based top-sold unique products recommendation for a specific user
user_id = 386730
top_sold_products_user = recommend_top_sold_products_by_category(df, user_id, 5)
print(f"Top Sold Products Recommendation for User {user_id}:")
print(top_sold_products_user)

Top Sold Products Recommendation for all clients:
        PROPOSITION  total_spend_on_product   PRICE  quantity_sold
454409       627917              3021364.30  1.5972   1.891663e+06
268327       366526              1453955.53  2.3100   6.294180e+05
585285       802428               344873.32  0.5940   5.805948e+05
395408       543679               330177.38  0.6336   5.211133e+05
666352       913550              1224886.79  2.5740   4.758690e+05
Top Sold Products Recommendation for User 386730:
      PROPOSITION  total_spend_on_product    PRICE  quantity_sold
684        459375              1246962.33   2.8380  439380.665962
59          33302              1340538.00   4.2900  312479.720280
451        241435               209175.14  12.9360   16170.001546
1275       868798               181983.30  13.1340   13855.893102
1224       868328                81947.23  14.5068    5648.883972


In [2]:
import pandas as pd
# read the processed_events_final file
df = pd.read_csv('processed_data/processed_events_final.csv', delimiter='|')





/var/folders/f3/ykt1vfz96xz48try_trwdn440000gn/T/ipykernel_11341/1465218501.py:3: DtypeWarning: Columns (6,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('processed_data/processed_events_final.csv', delimiter='|')


In [ ]:
import os
all_clients = [client_nr for client_nr in os.listdir('client_data')]
few_clients